## CabEdge Prediction Notebook

In [1]:
import pandas as pd
import numpy as np

import datetime as pydt

import seaborn as sns
import matplotlib.pyplot as plt

In [24]:
import dill

In [2]:
from fbprophet import Prophet

In [3]:
from fbprophet.diagnostics import cross_validation

In [4]:
import import_ipynb
import cabedge_utils

importing Jupyter notebook from cabedge_utils.ipynb


In [31]:
pip freeze | grep -i 'dill'

dill==0.3.1.1
Note: you may need to restart the kernel to use updated packages.


In [5]:
%store -r dropoff_model_object_list

In [6]:
%store -r pick_up_model_object_list

In [7]:
zone_distance_matrix = pd.read_csv('/home/jupyter/capstone/database/zone_distance_matrix.csv')
zone_df = pd.read_csv('/home/jupyter/capstone/database/zone_df.csv')

### Configuring Prediction algorithms 

### Function to find out the closest 10 zones based on Euclidean Distance

In [8]:
def closest_ten_zones(current_zone,zone_distance_matrix):
    best_zone_distance_list = []
    zone_demand_list =[]
    for zone in range(1,264):
        dist = zone_distance_matrix.loc[current_zone][zone]
        best_zone_distance_list.append(dist)
        zone_demand_list.append(zone)
    
    close_zone_distance = pd.DataFrame()
    close_zone_distance['zone'] = zone_demand_list
    close_zone_distance['Distance'] = best_zone_distance_list
    closest_zones = close_zone_distance.sort_values(by='Distance').iloc[0:9,0].values
    
    return closest_zones
      

### Function to predict number of pickups in those 10 zones

In [32]:
def predict_nb_pickup(test,closest_zones,pick_up_model_object_list):
    
    zone_pickup_dict = {}
    
    for pickup in closest_zones:
        predicted_pickup = pick_up_model_object_list[pickup-1].predict(test)['yhat'].values[0]
        if predicted_pickup < 0 :
            predicted_pickup = 0
    
        zone_pickup_dict[pickup] = predicted_pickup
    return zone_pickup_dict

### Function to predict number of dropoffs in those 10 zones

In [10]:
def predict_nb_dropoff(test,closest_zones,dropoff_model_object_list):
    
    zone_dropoff_dict = {}
    
    for dropoff in closest_zones:
        predicted_dropoff = dropoff_model_object_list[dropoff-1].predict(test)['yhat'].values[0]
        if predicted_dropoff < 0 :
            predicted_dropoff = 0
    
        zone_dropoff_dict[dropoff] = predicted_dropoff
    return zone_dropoff_dict

### Configuring Test Data

In [11]:
date_str = "2019-06-23 22"

In [12]:
date_object = pydt.datetime.strptime(date_str,'%Y-%m-%d %H')

In [13]:
print(date_object)

2019-06-23 22:00:00


In [14]:
test_1 = pd.DataFrame(columns=['ds','apparentTemperature','windSpeed','floor','cap'])
test_1.loc[0,'ds'] = date_object
test_1.loc[0,'apparentTemperature'] = 14.25
test_1.loc[0,'windSpeed'] = 1.56
test_1.loc[0,'floor'] = 0
test_1.loc[0, 'cap'] = 1000

test_1['apparentTemperature'] = test_1['apparentTemperature'].apply(float)
test_1['windSpeed'] = test_1['windSpeed'].apply(float)
test_1['cap'] = test_1['cap'].apply(int)
test_1['floor'] = test_1['floor'].apply(int)

In [15]:
test_1['ds'] = pd.to_datetime(test_1['ds'])

In [16]:
test_1

,ds,apparentTemperature,windSpeed,floor,cap
0,2019-06-23 22:00:00,14.25,1.56,0,1000


In [21]:
current_zone = 89
zone = [i for i in range(1,264)]
zone_distance_matrix.set_index('zone',inplace=True)
zone_distance_matrix.columns = zone

In [22]:
# %%snakeviz
closest_zones = closest_ten_zones(current_zone,zone_distance_matrix)
print(closest_zones)
zone_pickup_dict = predict_nb_pickup(test_1,closest_zones,pick_up_model_object_list)
zone_dropoff_dict = predict_nb_dropoff(test_1,closest_zones,dropoff_model_object_list)
# print(zone_pickup_dict)
# print(zone_dropoff_dict)
zone_demand_list = cabedge_utils.model_demand_supply(zone_pickup_dict,zone_dropoff_dict,current_zone)
# print(zone_demand_list)
best_zone = cabedge_utils.find_best_zone_distance(zone_distance_matrix,current_zone,zone_demand_list)
borough,best_zone_name = cabedge_utils.best_zone_func(best_zone)
print('CabEdge suggests you to drive to ',borough,best_zone_name)

[ 89  85 133 257 188 165 190  71  62]
Current Zone:  89 :  1.6178115081407924
[89] [0]
89
CabEdge suggests you to drive to  Brooklyn Flatbush/Ditmas Park


In [19]:
zone_distance_matrix.loc[239][151]

0.014849823328910338

In [20]:
zone_distance_matrix

,1,2,3,4,5,6,7,8,9,10,...,254,255,256,257,258,259,260,261,262,263
zone,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.352269,0.376308,0.200930,0.132704,0.148186,0.260545,0.269987,0.394234,0.394605,...,0.358806,0.225259,0.213953,0.203451,0.319342,0.376283,0.277612,0.166217,0.245715,0.245715
2,0.352269,0.000000,0.258903,0.188288,0.341424,0.222523,0.192947,0.195380,0.163953,0.096981,...,0.269489,0.161776,0.166297,0.150335,0.088195,0.292870,0.157767,0.207466,0.207369,0.207369
3,0.376308,0.258903,0.000000,0.193619,0.450332,0.338254,0.125603,0.115401,0.114679,0.187517,...,0.026314,0.184529,0.195438,0.246091,0.174105,0.038058,0.132748,0.226682,0.134761,0.134761
4,0.200930,0.188288,0.193619,0.000000,0.257024,0.146579,0.068065,0.078298,0.193308,0.200884,...,0.183885,0.028156,0.022133,0.069384,0.126837,0.206698,0.076891,0.035574,0.061511,0.061511
5,0.132704,0.341424,0.450332,0.257024,0.000000,0.123964,0.325036,0.335273,0.435119,0.411565,...,0.438666,0.271024,0.259068,0.220608,0.342416,0.460034,0.327995,0.223740,0.316014,0.316014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,0.376283,0.292870,0.038058,0.206698,0.460034,0.353218,0.140834,0.131235,0.152709,0.224937,...,0.024750,0.202440,0.212297,0.265449,0.206375,0.000000,0.156160,0.237096,0.145375,0.145375
260,0.277612,0.157767,0.132748,0.076891,0.327995,0.209862,0.036429,0.037617,0.116802,0.136629,...,0.131448,0.057039,0.069095,0.114397,0.073517,0.156160,0.000000,0.112307,0.053808,0.053808
261,0.166217,0.207466,0.226682,0.035574,0.223740,0.119593,0.101959,0.112152,0.228332,0.231470,...,0.215173,0.059226,0.048445,0.066700,0.156345,0.237096,0.112307,0.000000,0.092304,0.092304


In [63]:
pwd

'/home/jupyter/capstone'

## Creating pickle files

## Pickle files for Pickup 

In [25]:
# pick_up_model_object_list

In [29]:
zone = 1
for model in pick_up_model_object_list:
    filename = 'zone'+str(zone)+'.pkl'
    path_name = '/home/jupyter/capstone/pickup_pickle/' + filename
    print(path_name)
    with open(path_name, 'wb') as f:
        dill.dump(model, f)
        f.close()
    zone += 1

/home/jupyter/capstone/pickup_pickle/zone1.pkl
/home/jupyter/capstone/pickup_pickle/zone2.pkl
/home/jupyter/capstone/pickup_pickle/zone3.pkl
/home/jupyter/capstone/pickup_pickle/zone4.pkl
/home/jupyter/capstone/pickup_pickle/zone5.pkl
/home/jupyter/capstone/pickup_pickle/zone6.pkl
/home/jupyter/capstone/pickup_pickle/zone7.pkl
/home/jupyter/capstone/pickup_pickle/zone8.pkl
/home/jupyter/capstone/pickup_pickle/zone9.pkl
/home/jupyter/capstone/pickup_pickle/zone10.pkl
/home/jupyter/capstone/pickup_pickle/zone11.pkl
/home/jupyter/capstone/pickup_pickle/zone12.pkl
/home/jupyter/capstone/pickup_pickle/zone13.pkl
/home/jupyter/capstone/pickup_pickle/zone14.pkl
/home/jupyter/capstone/pickup_pickle/zone15.pkl
/home/jupyter/capstone/pickup_pickle/zone16.pkl
/home/jupyter/capstone/pickup_pickle/zone17.pkl
/home/jupyter/capstone/pickup_pickle/zone18.pkl
/home/jupyter/capstone/pickup_pickle/zone19.pkl
/home/jupyter/capstone/pickup_pickle/zone20.pkl
/home/jupyter/capstone/pickup_pickle/zone21.pkl
/

### Pickle files for for Dropoff

In [30]:
zone = 1
for model in dropoff_model_object_list:
    filename = 'zone'+str(zone)+'.pkl'
    path_name = '/home/jupyter/capstone/dropoff_pickle/' + filename
    print(path_name)
    with open(path_name, 'wb') as f:
        dill.dump(model, f)
        f.close()
    zone += 1

/home/jupyter/capstone/dropoff_pickle/zone1.pkl
/home/jupyter/capstone/dropoff_pickle/zone2.pkl
/home/jupyter/capstone/dropoff_pickle/zone3.pkl
/home/jupyter/capstone/dropoff_pickle/zone4.pkl
/home/jupyter/capstone/dropoff_pickle/zone5.pkl
/home/jupyter/capstone/dropoff_pickle/zone6.pkl
/home/jupyter/capstone/dropoff_pickle/zone7.pkl
/home/jupyter/capstone/dropoff_pickle/zone8.pkl
/home/jupyter/capstone/dropoff_pickle/zone9.pkl
/home/jupyter/capstone/dropoff_pickle/zone10.pkl
/home/jupyter/capstone/dropoff_pickle/zone11.pkl
/home/jupyter/capstone/dropoff_pickle/zone12.pkl
/home/jupyter/capstone/dropoff_pickle/zone13.pkl
/home/jupyter/capstone/dropoff_pickle/zone14.pkl
/home/jupyter/capstone/dropoff_pickle/zone15.pkl
/home/jupyter/capstone/dropoff_pickle/zone16.pkl
/home/jupyter/capstone/dropoff_pickle/zone17.pkl
/home/jupyter/capstone/dropoff_pickle/zone18.pkl
/home/jupyter/capstone/dropoff_pickle/zone19.pkl
/home/jupyter/capstone/dropoff_pickle/zone20.pkl
/home/jupyter/capstone/dropof